In [32]:
# ==========================================
# EXPERIMENT stage 3 – Test models with data augmentation (EasyEnsemble)
# ==========================================

from datetime import date

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

from imblearn.ensemble import EasyEnsembleClassifier
from collections import Counter

# MODELS
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

In [33]:
# Load data from CSV file
df = pd.read_csv("creditcard_dataset.csv")
print("Rozkład klas oryginalny:", Counter(df['Class']))

Rozkład klas oryginalny: Counter({0: 284315, 1: 492})


In [34]:
# Data separation to features and labels
X = df.drop('Class', axis=1)
y = df['Class']

In [35]:
# Splitting data for training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print("Rozkład klas w zbiorze treningowym:", Counter(y_train))

Rozkład klas w zbiorze treningowym: Counter({0: 227451, 1: 394})


In [36]:
# Create EasyEnsemble classifier
easy_ensemble = EasyEnsembleClassifier(n_estimators=10, random_state=42)


easy_ensemble .fit(X_train, y_train)

,n_estimators,10
,estimator,None
,warm_start,False
,sampling_strategy,'auto'
,replacement,False
,n_jobs,None
,random_state,42
,verbose,0


In [37]:

# Wynik EasyEnsemble (sam classifier)
y_pred_easy_ensemble = easy_ensemble.predict(X_test)

print("\n=== Wyniki EasyEnsembleClassifier ===")
print(confusion_matrix(y_test, y_pred_easy_ensemble))
print(classification_report(y_test, y_pred_easy_ensemble, target_names=["Correct", "Frauds"]))


=== Wyniki EasyEnsembleClassifier ===
[[54820  2044]
 [    8    90]]
              precision    recall  f1-score   support

     Correct       1.00      0.96      0.98     56864
      Frauds       0.04      0.92      0.08        98

    accuracy                           0.96     56962
   macro avg       0.52      0.94      0.53     56962
weighted avg       1.00      0.96      0.98     56962



In [38]:
# Models definition
models = {
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1),
    "XGBoost": XGBClassifier(eval_metric='logloss', random_state=42, n_jobs=-1),
    "Logistic Regression": LogisticRegression(max_iter=5000, class_weight='balanced', random_state=42, solver='lbfgs'),
    "Linear SVC": LinearSVC(class_weight='balanced', max_iter=2000, random_state=42)
}

In [39]:
# Training and model validation after EasyEnsemble
results = {}

for name, model in models.items():
    print(f"\nModel: {name}")
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    print(f"=== Results for model {name} ===")
    print(confusion_matrix(y_test, y_pred))
    report = classification_report(y_test, y_pred, target_names=["Correct", "Frauds"])
    print(report)

    results[name] = report


Model: Random Forest
=== Results for model Random Forest ===
[[56859     5]
 [   18    80]]
              precision    recall  f1-score   support

     Correct       1.00      1.00      1.00     56864
      Frauds       0.94      0.82      0.87        98

    accuracy                           1.00     56962
   macro avg       0.97      0.91      0.94     56962
weighted avg       1.00      1.00      1.00     56962


Model: XGBoost
=== Results for model XGBoost ===
[[56852    12]
 [   20    78]]
              precision    recall  f1-score   support

     Correct       1.00      1.00      1.00     56864
      Frauds       0.87      0.80      0.83        98

    accuracy                           1.00     56962
   macro avg       0.93      0.90      0.91     56962
weighted avg       1.00      1.00      1.00     56962


Model: Logistic Regression


c:\Users\lzloc\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 5000 iteration(s) (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=5000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


=== Results for model Logistic Regression ===
[[55435  1429]
 [    8    90]]
              precision    recall  f1-score   support

     Correct       1.00      0.97      0.99     56864
      Frauds       0.06      0.92      0.11        98

    accuracy                           0.97     56962
   macro avg       0.53      0.95      0.55     56962
weighted avg       1.00      0.97      0.99     56962


Model: Linear SVC
=== Results for model Linear SVC ===
[[55695  1169]
 [    8    90]]
              precision    recall  f1-score   support

     Correct       1.00      0.98      0.99     56864
      Frauds       0.07      0.92      0.13        98

    accuracy                           0.98     56962
   macro avg       0.54      0.95      0.56     56962
weighted avg       1.00      0.98      0.99     56962



In [41]:
# Saving results to file
dataset_description = "EasyEnsemble"
today = date.today()
file_name = "v3_results_" + dataset_description + ".txt"
with open(file_name, "w", encoding="utf-8") as f:
    f.write(f"Report created at : {today}\n")
    f.write(f"Comment           : EasyEnsemble augmentation\n")
    f.write(f"Rozkład klas w zbiorze treningowym (EasyEnsemble): {Counter(y_train)}\n")
    for name, report in results.items():
        f.write(f"\n=== {name} ===\n")
        f.write(report)
        f.write("\n" + "="*60 + "\n")

print("\n Experiment with dataset: " + dataset_description + " finished - see results in file: " + file_name)


 Experiment with dataset: EasyEnsemble finished - see results in file: v3_results_EasyEnsemble.txt
